<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Общее-впечатление" data-toc-modified-id="Общее-впечатление-0.1"><span class="toc-item-num">0.1&nbsp;&nbsp;</span><font color="orange">Общее впечатление</font></a></span></li><li><span><a href="#Общее-впечатление-(ревью-2)" data-toc-modified-id="Общее-впечатление-(ревью-2)-0.2"><span class="toc-item-num">0.2&nbsp;&nbsp;</span><font color="orange">Общее впечатление (ревью 2)</font></a></span></li></ul></li><li><span><a href="#Загрузка-и-подготовка-данных" data-toc-modified-id="Загрузка-и-подготовка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка и подготовка данных</a></span></li><li><span><a href="#Обучение-и-проверка-модели" data-toc-modified-id="Обучение-и-проверка-модели-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение и проверка модели</a></span></li><li><span><a href="#Подготовка-к-расчёту-прибыли" data-toc-modified-id="Подготовка-к-расчёту-прибыли-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Подготовка к расчёту прибыли</a></span><ul class="toc-item"><li><span><a href="#Вывод" data-toc-modified-id="Вывод-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Вывод</a></span></li></ul></li><li><span><a href="#Функция-для-расчёта-прибыли-по-выбранным-скважинам-и-предсказаниям-модели" data-toc-modified-id="Функция-для-расчёта-прибыли-по-выбранным-скважинам-и-предсказаниям-модели-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Функция для расчёта прибыли по выбранным скважинам и предсказаниям модели</a></span></li><li><span><a href="#Расчёт-прибыли-и-рисков" data-toc-modified-id="Расчёт-прибыли-и-рисков-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Расчёт прибыли и рисков</a></span><ul class="toc-item"><li><span><a href="#Вывод" data-toc-modified-id="Вывод-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>Вывод</a></span></li></ul></li><li><span><a href="#Вывод" data-toc-modified-id="Вывод-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Вывод</a></span></li><li><span><a href="#Чек-лист-готовности-проекта" data-toc-modified-id="Чек-лист-готовности-проекта-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Чек-лист готовности проекта</a></span></li></ul></div>

# Выбор локации для скважины

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

## Загрузка и подготовка данных

In [8]:
import pandas as pd 
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler

In [9]:
data_1 = pd.read_csv(r'C:\\Users\\vasni\\for git\\geo_data_1.csv')
display(data_1.head())

,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305


In [10]:
data = {}
for i in range(3):
    data[f'{i+1}_region'] = pd.read_csv(f'C:\\Users\\vasni\\for git\\geo_data_{i}.csv')
    display(data[f'{i+1}_region'].sample(n=5, random_state = 42))
    display(data[f'{i+1}_region'].isna().mean())
    #display(data[f'{i}_region'].nunique())
    display(data[f'{i+1}_region'].shape)
    display(data[f'{i+1}_region'].info())
    display(data[f'{i+1}_region'].describe())

,id,f0,f1,f2,product
75721,7C6bA,0.599283,-0.557623,2.121187,122.073350
80184,dVxsn,0.739017,-0.463156,-1.347584,48.738540
19864,1Gm3l,1.422743,-0.534917,3.718798,131.338088
76699,Nzg1t,1.580244,-0.238458,2.805149,88.327757
92991,Z7b4F,0.918974,0.023961,2.598575,36.959266


id         0.0
f0         0.0
f1         0.0
f2         0.0
product    0.0
dtype: float64

(100000, 5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


None

...

id         0.0
f0         0.0
f1         0.0
f2         0.0
product    0.0
dtype: float64

(100000, 5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


None

,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.002023,-0.002081,2.495128,95.000000
std,1.732045,1.730417,3.473445,44.749921
min,-8.760004,-7.084020,-11.970335,0.000000
25%,-1.162288,-1.174820,0.130359,59.450441
50%,0.009424,-0.009482,2.484236,94.925613
75%,1.158535,1.163678,4.858794,130.595027
max,7.238262,7.844801,16.739402,190.029838


<div class="alert alert-info">
<h1>Комментарий студента <a class="tocSkip"></a></h1>
Тип данных везде корректный. Пропущенных значений нет.Названия столбцов в одном регистре. Видно, что стандартное отлонение разнится по признакам и по дататестам, и нет четкой закономерности, например - в первом это f1 = .05, f2 =3.2, а в втором f1 = 5.11, f2 =1.7. 

Столбец "id" нам не нужен, можно удалить.
</div>

In [11]:
for i in data.keys():
    data[i] = data[i].drop(axis = 1, columns = 'id')

<div class="alert alert-info">
<h1>Комментарий студента <a class="tocSkip"></a></h1>
Данный дататест можно не проверять на мультиколлинеарность, так как все признаки имеют значение(по условию задачи) 
</div>

## Обучение и проверка модели

<div class="alert alert-info">
<h1>Комментарий студента <a class="tocSkip"></a></h1>
Разделим данные на признаки и целевой признак внутри словаря.
</div>

In [12]:
for i in list(data.keys()):
    data[f'{i}_features'] = data[i][['f0', 'f1', 'f2']]
    data[f'{i}_target'] = data[i]['product']

In [13]:
data.keys()

dict_keys(['1_region', '2_region', '3_region', '1_region_features', '1_region_target', '2_region_features', '2_region_target', '3_region_features', '3_region_target'])

<div class="alert alert-info">
<h1>Комментарий студента <a class="tocSkip"></a></h1>
Разделим на обучающую и валидационную выборки.
</div>

In [14]:
region_1_features_train, region_1_features_valid, region_1_target_train, region_1_target_valid = train_test_split(
    data['1_region_features'], data['1_region_target'], test_size = 0.25, random_state = 42)

region_2_features_train, region_2_features_valid, region_2_target_train, region_2_target_valid = train_test_split(
    data['2_region_features'], data['2_region_target'], test_size = 0.25, random_state = 42)

region_3_features_train, region_3_features_valid, region_3_target_train, region_3_target_valid = train_test_split(
    data['3_region_features'], data['3_region_target'], test_size = 0.25, random_state = 42)

In [15]:
scaler = StandardScaler()
def standart(features_train, features_valid):
    scaler.fit(features_train)
    features_train_scaled = scaler.transform(features_train)
    features_valid_scaled = scaler.transform(features_valid) 
    return features_train_scaled, features_valid_scaled

<div class="alert alert-info">
<h1>Комментарий студента <a class="tocSkip"></a></h1>
Отмасштабируем признаки.
</div>

In [16]:
region_1_features_train, region_1_features_valid = standart(region_1_features_train, region_1_features_valid)

region_2_features_train, region_2_features_valid = standart(region_2_features_train, region_2_features_valid)

region_3_features_train, region_3_features_valid = standart(region_3_features_train, region_3_features_valid)

In [17]:
model = LinearRegression()

In [22]:
def model_sensei(features_train, features_valid, target_train, target_valid):
    model.fit(features_train, target_train)
    predictions = model.predict(features_valid)
    predictions = pd.Series(predictions, index=target_valid.index)
    result = mean_squared_error(target_valid, predictions)
    print("MSE модели линейной регрессии на валидационной выборке:", result.round(2))
    print("RMSE модели линейной регрессии на валидационной выборке:", (result**0.5).round(2))
    print('Средний запас предсказанного сырья', round(predictions.mean(), 2))
    return predictions

In [25]:
predictions_1 = model_sensei(region_1_features_train, region_1_features_valid, region_1_target_train, region_1_target_valid)

MSE модели линейной регрессии на валидационной выборке: 1425.56
RMSE модели линейной регрессии на валидационной выборке: 37.76
Средний запас предсказанного сырья 92.4


In [26]:
predictions_2 = model_sensei(region_2_features_train, region_2_features_valid, region_2_target_train, region_2_target_valid)

MSE модели линейной регрессии на валидационной выборке: 0.79
RMSE модели линейной регрессии на валидационной выборке: 0.89
Средний запас предсказанного сырья 68.71


In [27]:
predictions_3 = model_sensei(region_3_features_train, region_3_features_valid, region_3_target_train, region_3_target_valid)

MSE модели линейной регрессии на валидационной выборке: 1611.69
RMSE модели линейной регрессии на валидационной выборке: 40.15
Средний запас предсказанного сырья 94.77


In [28]:
table = ({'mse_1': 1425.56,
'mse_2': 0.79,
'mse_3': 1611.69,
'avr_1': 92.4,
'avr_2': 68.71,
'avr_3': 94.77})

## Подготовка к расчёту прибыли

<div class="alert alert-info">
<h1>Комментарий студента <a class="tocSkip"></a></h1>
Определим нужные нам переменные.<br />
places_count: кол-во исследуемыз точек<br />
    <br />
budget: бюджет<br />
    <br />
a_barrel_price: цена за один баррель<br />
    <br />
product_price: цена за 1000 баррелей(единицу целевого признака)<br />
    <br />
max_loss_probability: максимально допустимая веротяность убытков
</div>

In [29]:
places_count = 500

the_best_places = 200

budget = 10**10

a_barrel_price = 450

product_price = a_barrel_price * 1000

max_loss_probability = 2.5

<div class="alert alert-block alert-info">
<b>Совет: </b> В названиях константных переменных лучше использовать только БОЛЬШИЕ буквы. Это соглашение между программистами.
</div>

<div class="alert alert-info">
<h1>Комментарий студента <a class="tocSkip"></a></h1>
Почитаем средний объем по каждому объекту.
</div>

In [31]:
for i in data.keys():
    if 'target' in i:
        display(i)
        display(round(data[i].mean(), 2))

'1_region_target'

92.5

'2_region_target'

68.83

'3_region_target'

95.0

<div class="alert alert-info">
<h1>Комментарий студента <a class="tocSkip"></a></h1>
Достаточный объем для безубыточной разработки.
</div>

In [32]:
round(budget / (product_price * 200), 2)

111.11

### Вывод

<div class="alert alert-info">
<h1>Комментарий студента <a class="tocSkip"></a></h1>
Комментарий студента:</b> В результате расчета средней прибыли по регионам и достаточного объема сырья для безубыточной разрбаотки скважины можно сделать вывод, что выбирать скважины случайным образом нецелесообразно, так как даже на этапе предварительной оценки проект получается убыточным.
</div>

## Функция для расчёта прибыли по выбранным скважинам и предсказаниям модели

<div class="alert alert-info">
<h1>Комментарий студента <a class="tocSkip"></a></h1>
Функция для расчета прибыли
</div>

In [33]:
def income(predictions, target, count):
    #target = target.reset_index(drop=True)
    #predictions = predictions.reset_index(drop=True)
    predictions = predictions.sort_values(ascending=False)
    sum_ = target[predictions.index][:count].sum()
    return sum_ * product_price - budget

In [34]:
income(predictions_1, region_1_target_valid, 200)

3359141114.462179

In [35]:
income(predictions_2, region_2_target_valid, 200)

2415086696.681511

In [36]:
income(predictions_3, region_3_target_valid, 200)

2598571759.374111

## Расчёт прибыли и рисков 

In [37]:
state = np.random.RandomState(42)
count = 200

def bootstrap(predictions, target, count): 
    target = target.where(target > 0)
    values = []
    for i in range(1000):
        subsample_target = target.sample(n=500, replace=True, random_state=state)
        subsample_predictions = predictions[subsample_target.index]
        values.append(income(subsample_predictions, subsample_target, count))
    values = pd.Series(values)
    lower = values.quantile(0.025)
    upper = values.quantile(0.975)
    display('Средняя прибыль', values.mean())
    display('Нижняя граница доверительного интервала',lower)
    display('Верхняя граница доверительного интервала', upper)
    return values

In [38]:
values_1 = bootstrap(predictions_1, region_1_target_valid, count)

'Средняя прибыль'

427847560.4625241

'Нижняя граница доверительного интервала'

-97249829.56859529

'Верхняя граница доверительного интервала'

954215192.708815

In [39]:
values_2 = bootstrap(predictions_2, region_2_target_valid, count)

'Средняя прибыль'

511530218.1485253

'Нижняя граница доверительного интервала'

91700564.13644528

'Верхняя граница доверительного интервала'

921455668.3285091

In [40]:
values_3 = bootstrap(predictions_3, region_3_target_valid, count)

'Средняя прибыль'

408545681.21493495

'Нижняя граница доверительного интервала'

-120624872.94271684

'Верхняя граница доверительного интервала'

960859440.7253834

### Вывод 

In [41]:
final_table = pd.DataFrame({
    'Показатель':['MSE', 'RMSE', 'Средний запас', 'Средняя прибыль(млн)', 'Вероятность убытка'],
    'Регион_1':[table['mse_1'], table['mse_1']**0.5, data['1_region_target'].mean(), values_1.mean()/10**6, (values_1 < 0).mean()],
    'Регион_2':[table['mse_2'], table['mse_2']**0.5, data['2_region_target'].mean(), values_2.mean()/10**6, (values_2 < 0).mean()],
    'Регион_3':[table['mse_3'], table['mse_3']**0.5, data['3_region_target'].mean(), values_3.mean()/10**6, (values_3 < 0).mean()]
})

In [42]:
display(final_table)

,Показатель,Регион_1,Регион_2,Регион_3
0,MSE,1425.560000,0.790000,1611.690000
1,RMSE,37.756589,0.888819,40.145859
2,Средний запас,92.500000,68.825000,95.000000
3,Средняя прибыль(млн),427.847560,511.530218,408.545681
4,Вероятность убытка,0.055000,0.006000,0.075000


In [47]:
final_table_1 = pd.DataFrame({
    'Metrics':['MSE', 'RMSE', 'Average stock', 'Average profit(million)', 'Probability of loss'],
    'Region_1':[table['mse_1'], table['mse_1']**0.5, data['1_region_target'].mean(), values_1.mean()/10**6, (values_1 < 0).mean()],
    'Region_2':[table['mse_2'], table['mse_2']**0.5, data['2_region_target'].mean(), values_2.mean()/10**6, (values_2 < 0).mean()],
    'Region_3':[table['mse_3'], table['mse_3']**0.5, data['3_region_target'].mean(), values_3.mean()/10**6, (values_3 < 0).mean()]
})

In [48]:
display(final_table_1)

,Metrics,Region_1,Region_2,Region_3
0,MSE,1425.560000,0.790000,1611.690000
1,RMSE,37.756589,0.888819,40.145859
2,Average stock,92.500000,68.825000,95.000000
3,Average profit(million),427.847560,511.530218,408.545681
4,Probability of loss,0.055000,0.006000,0.075000


## Вывод

<div class="alert alert-block alert-warning">
<b>Комментарий студента:</b> В ходе исследования модель линейной регрессии была обучена на тренировочной выборке.<br /> 
       На основании ее предсказаний был подсчитан средний доход, а так же определен достаточный объём сырья для безубыточной разработки новой скважины, из этих расчтов был сделан вывод, что выбирать скважины вслепую нецелесообразно, так как проект будет убыточным.<br />
    <br /> 
        С помощью предсказаний модели и технологии bootstrap была подсчитана средняя прибыль и веротятсноть убытков, на основании результатов этих расчетов Регион №2 является наиболее переспективным для разработки, так как показал наименьший процент вероятности убытков(0.6 %) наибольшую среднюю прибыль(511.53 млн.)
</div>